# Min Cost Flow problem

In [ ]:
import gurobipy as grb
import pandas as pd
import numpy as np
import os

%matplotlib notebook
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

from shapely.geometry import Point, LineString
import geopandas as gpd

## NYC Subway network

In [ ]:
thepath = os.getcwd().split("veteran_students_mec_optim\\Pauline")[0]
arcs = pd.read_csv(thepath + "\\data_mec_optim\\networks_subway\\NYC\\arcs.csv", sep=',').sort_values(by=['route_id'])

nodes = pd.read_csv(thepath + "\\data_mec_optim\\networks_subway\\NYC\\nodes.csv", sep=',')

Stations' caracteristics are contained in nodes dataframe:

In [ ]:
nodes.head()

Routes between stations are contained in arcs dataframe:

In [ ]:
arcs.head()

Beware of false duplicates! ine M and J run in parallel. In this simple application we keep only line M.

In [ ]:
pd.concat(g for _, g in arcs.groupby(["from_stop_id","to_stop_id"]) if len(g) > 1)

In [ ]:
arcs.insert(7,"duplicates",arcs.duplicated(["from_stop_id","to_stop_id"]))
todrop = arcs[(arcs['duplicates']) & (arcs['route_id']=='M')].index
arcs.drop(todrop, inplace=True)
any(arcs.duplicated(["from_stop_id","to_stop_id"]))

In [ ]:
print(len(nodes))
print(len(arcs))

In [ ]:
nb_nodes = arcs['from_stop_nb'].nunique()
names_nodes = nodes['stop_name'] + ' ' + nodes['route_id']
arcs_list = [(i, j) for i, j in zip(arcs['from_stop_nb'], arcs['to_stop_nb'])]
weights = arcs['dis_line'].values

In [ ]:
origin_node = 452
destination_node = 471

In [ ]:
m=grb.Model('NYC Subway')
paths = m.addVars(arcs_list, obj = weights, name='arcs')
m.addConstrs((paths.sum('*', station) - paths.sum(station, '*') == 0 for station in range(nb_nodes)
              if station not in [origin_node, destination_node]), name='Constr')
m.addConstr(paths.sum('*', origin_node) - paths.sum(origin_node, '*') == 1, name='Constr')
m.addConstr(paths.sum('*', destination_node) - paths.sum(destination_node, '*') == -1, name='Constr')

In [ ]:
m.optimize()
path_taken = origin_node
path_list = []
step = 1
if m.status == grb.GRB.Status.OPTIMAL:
    print('***Optimal solution***')
    print('Minimum distance from', names_nodes[origin_node - 1], 'to',
          names_nodes[destination_node - 1], '\n', m.objVal)
    print('0 :', names_nodes[origin_node - 1], '(#%d)' % origin_node)
    solution = m.getAttr('x', paths)
    while path_taken != destination_node:
        for arc in arcs_list:
            if arc[1] == path_taken and solution[arc] == 1:
                print(step, ':', names_nodes[arc[0] - 1], '(#%d)' % arc[0])
                path_taken = arc[0]
                path_list.append(path_taken)
                step += 1

In [ ]:
path_list.insert(0,origin_node)
path_list

In [ ]:
geometry_nodes = [Point(xy) for xy in zip(nodes['stop_lon'], nodes['stop_lat'])]
gdf_nodes = gpd.GeoDataFrame(nodes,geometry=geometry_nodes)
gdf_nodes.head()

In [ ]:
arcs_coord_int = pd.merge(arcs[['from_stop_id', 'to_stop_id']].rename(index=str, columns={'from_stop_id': 'stop_id'}), 
         gdf_nodes[['stop_id', 'geometry']].rename(index=str, columns={'geometry': 'from_geometry'}), 
         on = 'stop_id').rename(index=str, columns={'stop_id': 'from_stop_id'})

arcs_coord = pd.merge(arcs_coord_int.rename(index=str, columns={'to_stop_id': 'stop_id'}),
                     gdf_nodes[['stop_id', 'geometry']].rename(index=str, columns={'geometry': 'to_geometry'}),
                     on = 'stop_id').rename(index=str, columns={'stop_id': 'to_stop_id'})
del arcs_coord_int
arcs_coord.head()

In [ ]:
geometry_arcs = [LineString(xy) for xy in zip(arcs_coord['from_geometry'],arcs_coord['to_geometry'])]
gdf_arcs = gpd.GeoDataFrame(arcs,geometry=geometry_arcs)
gdf_arcs.head()

In [ ]:
def animate(i):
    label = 'timestep {0}'.format(i)
    print(label)
    # Update the line and the axes (with a new xlabel). Return a tuple of
    # "artists" that have to be redrawn for this frame.
    stop_id = gdf_arcs[gdf_arcs["to_stop_nb"]==path_list[i]]['to_stop_id'].values[0]
    stop_to_plot = gdf_nodes[gdf_nodes["stop_id"]==stop_id]

    ax.text(stop_to_plot['stop_lon'], stop_to_plot['stop_lat'],stop_to_plot['stop_name'].values[0], 
            size = 'medium', fontweight='bold')
    stop_to_plot.plot(marker = 'o', color = 'green', markersize=50, ax=ax)

    ax.set_xlabel(label)
    return ax

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
ax.set_xlim([-74.3, -73.7])
ax.set_ylim([40.5, 40.95])
ax.set_yticklabels([])
ax.set_xticklabels([])

gdf_arcs.plot(color = 'lightblue',ax=ax)
gdf_nodes.plot(marker = 'o', color = 'lightgreen', markersize=50, ax=ax)

In [ ]:
anim = FuncAnimation(fig, animate, frames = np.arange(0, len(path_list)), interval = 2000, repeat = False)
plt.show()